In [3]:
import glob,os
import jwst
from astropy.io import fits
print(f'JWST version: {jwst.__version__}')

root_dir = './test_mosaic'
filter_name = 'F560W'

# input dir and files
inputdir_level3 = f'{root_dir}/{filter_name}_level2_gaia'
filepattern = '*tweakregstep.fits'
inputfiles_level3 = sorted(glob.glob(f'{inputdir_level3}/{filepattern}'))
print(f'Found {len(inputfiles_level3)} input files, checking for the right filter...')
# check for correct filter
inputfiles_level3_filter = []
for filename in inputfiles_level3: 
    hdr = fits.getheader(filename)
    if hdr["FILTER"].lower() == filter_name.lower():
        print(f'KEEP {filename}: {hdr["FILTER"]}')
        inputfiles_level3_filter.append(os.path.abspath(filename))
    else:
        print(f'skipping {filename}: {hdr["FILTER"]}')
print(f'{len(inputfiles_level3_filter)} out of {len(inputfiles_level3)} have the correct filter')

# output dir, and make sure it exists
outdir_level3 = f'{root_dir}/{filter_name}_level3_gaia'
print(f'Outdir level3: {outdir_level3}')
if not os.path.isdir(outdir_level3):
    os.makedirs(outdir_level3)

JWST version: 1.8.0
Found 4 input files, checking for the right filter...
KEEP ./test_mosaic/F560W_level2_gaia/jw02666001001_02101_00001_mirimage_tweakregstep.fits: F560W
KEEP ./test_mosaic/F560W_level2_gaia/jw02666001001_02101_00002_mirimage_tweakregstep.fits: F560W
KEEP ./test_mosaic/F560W_level2_gaia/jw02666001001_02101_00003_mirimage_tweakregstep.fits: F560W
KEEP ./test_mosaic/F560W_level2_gaia/jw02666001001_02101_00004_mirimage_tweakregstep.fits: F560W
4 out of 4 have the correct filter
Outdir level3: ./test_mosaic/F560W_level3_gaia


In [4]:
from jwst.associations import asn_from_list
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base

asn3 = asn_from_list.asn_from_list(inputfiles_level3_filter, rule=DMS_Level3_Base, product_name=filter_name)
miri_asn_file = f'{outdir_level3}/{filter_name}.json'
with open(miri_asn_file, 'w') as outfile:
    name, serialized = asn3.dump(format='json')
    outfile.write(serialized)
print(asn3)

jwnoprogram-a3001_none_003_asn with 1 products
Rule=DMS_Level3_Base
No constraints
Products:
	F560W with 4 members


In [5]:
from jwst.pipeline import calwebb_image3

image3 = calwebb_image3.Image3Pipeline()

2022-12-14 12:55:30,061 - stpipe.Image3Pipeline - INFO - Image3Pipeline instance created.
2022-12-14 12:55:30,062 - stpipe.Image3Pipeline.assign_mtwcs - INFO - AssignMTWcsStep instance created.
2022-12-14 12:55:30,064 - stpipe.Image3Pipeline.tweakreg - INFO - TweakRegStep instance created.
2022-12-14 12:55:30,066 - stpipe.Image3Pipeline.skymatch - INFO - SkyMatchStep instance created.
2022-12-14 12:55:30,069 - stpipe.Image3Pipeline.outlier_detection - INFO - OutlierDetectionStep instance created.
2022-12-14 12:55:30,072 - stpipe.Image3Pipeline.resample - INFO - ResampleStep instance created.
2022-12-14 12:55:30,074 - stpipe.Image3Pipeline.source_catalog - INFO - SourceCatalogStep instance created.


In [6]:
image3.output_dir = outdir_level3
image3.save_results = True
### IMPORTANT!! you have to set image3.tweakreg.skip = True, otherwise tweakreg will screw up the WCS again!
image3.tweakreg.skip = True
#image3.tweakreg.align_to_gaia = True
image3.skymatch.skip = True
#image3.skymatch.skymethod = 'global+match'
image3.skymatch.match_down = False
image3.source_catalog.skip=False
#image3.skymatch.subtract = True

In [7]:
image3.run(miri_asn_file)

2022-12-14 12:55:37,077 - stpipe.Image3Pipeline - INFO - Step Image3Pipeline running with args ('./test_mosaic/F560W_level3_gaia/F560W.json',).
2022-12-14 12:55:37,089 - stpipe.Image3Pipeline - INFO - Step Image3Pipeline parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': './test_mosaic/F560W_level3_gaia', 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'steps': {'assign_mtwcs': {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': True, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': 'assign_mtwcs', 'search_output_file': True, 'input_dir': ''}, 'tweakreg': {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': True, 'output_use_index': True, 'save_results': False, 'skip': 

2022-12-14 12:55:45,995 - stpipe.Image3Pipeline.outlier_detection - INFO - Performing outlier detection on 4 inputs
2022-12-14 12:55:45,995 - stpipe.Image3Pipeline.outlier_detection - INFO - Driz parameter kernel: square
2022-12-14 12:55:45,996 - stpipe.Image3Pipeline.outlier_detection - INFO - Driz parameter pixfrac: 1.0
2022-12-14 12:55:45,997 - stpipe.Image3Pipeline.outlier_detection - INFO - Driz parameter fillval: INDEF
2022-12-14 12:55:45,997 - stpipe.Image3Pipeline.outlier_detection - INFO - Driz parameter weight_type: ivm
2022-12-14 12:55:45,998 - stpipe.Image3Pipeline.outlier_detection - INFO - Output pixel scale ratio: 1.0
2022-12-14 12:55:46,185 - stpipe.Image3Pipeline.outlier_detection - INFO - 1 exposures to drizzle together
2022-12-14 12:55:46,774 - stpipe.Image3Pipeline.outlier_detection - INFO - Drizzling (1024, 1032) --> (1172, 1178)
2022-12-14 12:55:47,129 - stpipe.Image3Pipeline.outlier_detection - INFO - Exposure jw02666001001_02101_00001_mirimage_outlier_i2d.fits s

2022-12-14 12:56:13,678 - stpipe.Image3Pipeline.source_catalog - INFO - Using APCORR reference file: /grp/crds/cache/references/jwst/jwst_miri_apcorr_0008.fits
2022-12-14 12:56:13,710 - stpipe.Image3Pipeline.source_catalog - INFO - Using ABVEGAOFFSET reference file: /grp/crds/cache/references/jwst/jwst_miri_abvegaoffset_0001.asdf
2022-12-14 12:56:13,712 - stpipe.Image3Pipeline.source_catalog - INFO - Instrument: MIRI
2022-12-14 12:56:13,713 - stpipe.Image3Pipeline.source_catalog - INFO - Detector: MIRIMAGE
2022-12-14 12:56:13,714 - stpipe.Image3Pipeline.source_catalog - INFO - Filter: F560W
2022-12-14 12:56:13,714 - stpipe.Image3Pipeline.source_catalog - INFO - Subarray: FULL
2022-12-14 12:56:13,935 - stpipe.Image3Pipeline.source_catalog - INFO - AB to Vega magnitude offset 3.76066
2022-12-14 12:56:14,029 - stpipe.Image3Pipeline.source_catalog - INFO - Background could not be estimated in meshes. Using the entire unmasked array for background estimation: bkg_boxsize=(1172, 1178).
2022-

In [8]:
from jwst.source_catalog import SourceCatalogStep
import re

snr_thresholds = [5.0,3.0,2.0]
#snr_thresholds = [2.0]
npixel_vals = [15,10]



files = glob.glob(f'{outdir_level3}/{filter_name}_i2d.fits')

for file in files:
    for snr in snr_thresholds:
        for npix in npixel_vals:
            outfile = re.sub('_([a-zA-Z0-9]+)\.fits$','',file)
            outfile += f'_snr{snr:.0f}_npix{npix:.0f}_cat.ecsv'
            print(f'Running {outfile}')
            result = SourceCatalogStep.call(file, 
                                            snr_threshold=float(snr),
                                            npixels = int(npix), bkg_boxsize = 100,
                                            output_file=outfile, 
                                            save_results=True,output_dir=f'{outdir_level3}')

2022-12-14 12:58:40,262 - stpipe - WARNING - <>:15: DeprecationWarning: invalid escape sequence '\.'

2022-12-14 12:58:40,265 - stpipe - WARNING - <>:15: DeprecationWarning: invalid escape sequence '\.'

2022-12-14 12:58:40,266 - stpipe - WARNING - /var/folders/16/91kljsb17fd5xnw3fr_fdj70000149/T/ipykernel_81315/3323345972.py:15: DeprecationWarning: invalid escape sequence '\.'
  outfile = re.sub('_([a-zA-Z0-9]+)\.fits$','',file)



Running ./test_mosaic/F560W_level3_gaia/F560W_snr5_npix15_cat.ecsv


2022-12-14 12:58:40,712 - stpipe - INFO - PARS-SOURCECATALOGSTEP parameters found: /grp/crds/cache/references/jwst/jwst_miri_pars-sourcecatalogstep_0020.asdf
2022-12-14 12:58:40,796 - stpipe.sourcecatalog - INFO - SourceCatalogStep instance created.
2022-12-14 12:58:40,933 - stpipe - WARNING - /Users/arest/anaconda3/envs/jwst/lib/python3.10/site-packages/stpipe/step.py:419: ResourceWarning: unclosed file <_io.BufferedReader name='./test_mosaic/F560W_level3_gaia/F560W_i2d.fits'>
  gc.collect()

2022-12-14 12:58:40,945 - stpipe.sourcecatalog - INFO - Step sourcecatalog running with args ('./test_mosaic/F560W_level3_gaia/F560W_i2d.fits',).
2022-12-14 12:58:40,948 - stpipe.sourcecatalog - INFO - Step sourcecatalog parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': '/Users/arest/nircam/jhat/test_mosaic/F560W_level3_gaia/F560W_snr5_npix15_cat.ecsv', 'output_dir': './test_mosaic/F560W_level3_gaia', 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'sa

Running ./test_mosaic/F560W_level3_gaia/F560W_snr5_npix10_cat.ecsv


2022-12-14 12:58:42,666 - stpipe - INFO - PARS-SOURCECATALOGSTEP parameters found: /grp/crds/cache/references/jwst/jwst_miri_pars-sourcecatalogstep_0020.asdf
2022-12-14 12:58:42,696 - stpipe.sourcecatalog - INFO - SourceCatalogStep instance created.
2022-12-14 12:58:42,802 - stpipe - WARNING - /Users/arest/anaconda3/envs/jwst/lib/python3.10/site-packages/stpipe/step.py:419: ResourceWarning: unclosed file <_io.BufferedReader name='./test_mosaic/F560W_level3_gaia/F560W_i2d.fits'>
  gc.collect()

2022-12-14 12:58:42,812 - stpipe.sourcecatalog - INFO - Step sourcecatalog running with args ('./test_mosaic/F560W_level3_gaia/F560W_i2d.fits',).
2022-12-14 12:58:42,815 - stpipe.sourcecatalog - INFO - Step sourcecatalog parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': '/Users/arest/nircam/jhat/test_mosaic/F560W_level3_gaia/F560W_snr5_npix10_cat.ecsv', 'output_dir': './test_mosaic/F560W_level3_gaia', 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'sa

...

2022-12-14 12:58:46,465 - stpipe - INFO - PARS-SOURCECATALOGSTEP parameters found: /grp/crds/cache/references/jwst/jwst_miri_pars-sourcecatalogstep_0020.asdf
2022-12-14 12:58:46,498 - stpipe.sourcecatalog - INFO - SourceCatalogStep instance created.
2022-12-14 12:58:46,644 - stpipe.sourcecatalog - INFO - Step sourcecatalog running with args ('./test_mosaic/F560W_level3_gaia/F560W_i2d.fits',).
2022-12-14 12:58:46,646 - stpipe.sourcecatalog - INFO - Step sourcecatalog parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': '/Users/arest/nircam/jhat/test_mosaic/F560W_level3_gaia/F560W_snr3_npix10_cat.ecsv', 'output_dir': './test_mosaic/F560W_level3_gaia', 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': 'cat', 'search_output_file': True, 'input_dir': '', 'bkg_boxsize': 100, 'kernel_fwhm': 2.0, 'snr_threshold': 3.0, 'npixels': 10, 'deblend': False, 'aperture_ee1': 30, 'aperture_ee2': 50, 'aperture_ee3': 70

Running ./test_mosaic/F560W_level3_gaia/F560W_snr2_npix15_cat.ecsv


2022-12-14 12:58:48,563 - stpipe - INFO - PARS-SOURCECATALOGSTEP parameters found: /grp/crds/cache/references/jwst/jwst_miri_pars-sourcecatalogstep_0020.asdf
2022-12-14 12:58:48,592 - stpipe.sourcecatalog - INFO - SourceCatalogStep instance created.
2022-12-14 12:58:48,728 - stpipe.sourcecatalog - INFO - Step sourcecatalog running with args ('./test_mosaic/F560W_level3_gaia/F560W_i2d.fits',).
2022-12-14 12:58:48,730 - stpipe.sourcecatalog - INFO - Step sourcecatalog parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': '/Users/arest/nircam/jhat/test_mosaic/F560W_level3_gaia/F560W_snr2_npix15_cat.ecsv', 'output_dir': './test_mosaic/F560W_level3_gaia', 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': 'cat', 'search_output_file': True, 'input_dir': '', 'bkg_boxsize': 100, 'kernel_fwhm': 2.0, 'snr_threshold': 2.0, 'npixels': 15, 'deblend': False, 'aperture_ee1': 30, 'aperture_ee2': 50, 'aperture_ee3': 70

Running ./test_mosaic/F560W_level3_gaia/F560W_snr2_npix10_cat.ecsv


2022-12-14 12:58:50,727 - stpipe - INFO - PARS-SOURCECATALOGSTEP parameters found: /grp/crds/cache/references/jwst/jwst_miri_pars-sourcecatalogstep_0020.asdf
2022-12-14 12:58:50,777 - stpipe.sourcecatalog - INFO - SourceCatalogStep instance created.
2022-12-14 12:58:50,899 - stpipe - WARNING - /Users/arest/anaconda3/envs/jwst/lib/python3.10/site-packages/stpipe/step.py:419: ResourceWarning: unclosed file <_io.BufferedReader name='./test_mosaic/F560W_level3_gaia/F560W_i2d.fits'>
  gc.collect()

2022-12-14 12:58:50,910 - stpipe.sourcecatalog - INFO - Step sourcecatalog running with args ('./test_mosaic/F560W_level3_gaia/F560W_i2d.fits',).
2022-12-14 12:58:50,913 - stpipe.sourcecatalog - INFO - Step sourcecatalog parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': '/Users/arest/nircam/jhat/test_mosaic/F560W_level3_gaia/F560W_snr2_npix10_cat.ecsv', 'output_dir': './test_mosaic/F560W_level3_gaia', 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'sa